In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [4]:
columns = [
    'movie_id', 'title', 'release_date', 'movie_popularity', 
    'vote_average', 'vote_count', 'budget', 'revenue', 'ratings', 'actor_name',
    'actor_id', 'character', 'actor_gender', 'actor_popularity', 'director_name', 'director_id', 
    'director_gender', 'director_popularity', 'studio_id', 'studio_name', 'origin_country'
]

target = ['revenue']

In [5]:
# Load the data
file_path = Path('meta.csv')
df = pd.read_csv(file_path)[:-2]
# , index_col=0
# df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

df.actor_gender[df.actor_gender == 2] = 'male'
df.actor_gender[df.actor_gender == 1] = 'female'

df.director_gender[df.director_gender == 2] = 'male'
df.director_gender[df.director_gender == 1] = 'female'

df

,movie_id,title,release_date,movie_popularity,vote_average,vote_count,budget,revenue,ratings,actor_name,...,actor_gender,character,actor_popularity,director_name,director_id,director_gender,director_popularity,studio_id,studio_name,origin_country
0,671,Harry Potter and the Philosopher's Stone,2001-11-16,305.856,7.9,21398,125000000,976475550,PG,Emma Watson,...,female,Hermione Granger,16.930,Chris Columbus,10965,male,2.589,436,1492 Pictures,US
1,671,Harry Potter and the Philosopher's Stone,2001-11-16,305.856,7.9,21398,125000000,976475550,PG,Emma Watson,...,female,Hermione Granger,16.930,Chris Columbus,10965,male,2.589,174,Warner Bros. Pictures,US
2,671,Harry Potter and the Philosopher's Stone,2001-11-16,305.856,7.9,21398,125000000,976475550,PG,Tom Felton,...,male,Draco Malfoy,10.918,Chris Columbus,10965,male,2.589,436,1492 Pictures,US
3,671,Harry Potter and the Philosopher's Stone,2001-11-16,305.856,7.9,21398,125000000,976475550,PG,Tom Felton,...,male,Draco Malfoy,10.918,Chris Columbus,10965,male,2.589,437,Heyday Films,GB
4,671,Harry Potter and the Philosopher's Stone,2001-11-16,305.856,7.9,21398,125000000,976475550,PG,Tom Felton,...,male,Draco Malfoy,10.918,Chris Columbus,10965,male,2.589,174,Warner Bros. Pictures,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1262,425909,Ghostbusters: Afterlife,2021-11-11,428.352,7.6,40,75000000,16000000,PG-13,Bill Murray,...,male,Dr. Peter Venkman,14.048,Jason Reitman,52443,male,4.105,5,Columbia Pictures,US
1263,425909,Ghostbusters: Afterlife,2021-11-11,428.352,7.6,40,75000000,16000000,PG-13,Bill Murray,...,male,Dr. Peter Venkman,14.048,Jason Reitman,52443,male,4.105,2364,The Montecito Picture Company,US
1264,425909,Ghostbusters: Afterlife,2021-11-11,428.352,7.6,40,75000000,16000000,PG-13,Bill Murray,...,male,Dr. Peter Venkman,14.048,Jason Reitman,52443,male,4.105,13240,Bron Studios,CA
1265,425909,Ghostbusters: Afterlife,2021-11-11,428.352,7.6,40,75000000,16000000,PG-13,Carrie Coon,...,female,Callie Spengler,11.375,Jason Reitman,52443,male,4.105,34,Sony Pictures,US


In [6]:
# More data cleaning
noID_df = df.drop(columns=['movie_id', 'actor_id', 'director_id', 'studio_id'])

noID_df

,title,release_date,movie_popularity,vote_average,vote_count,budget,revenue,ratings,actor_name,actor_gender,character,actor_popularity,director_name,director_gender,director_popularity,studio_name,origin_country
0,Harry Potter and the Philosopher's Stone,2001-11-16,305.856,7.9,21398,125000000,976475550,PG,Emma Watson,female,Hermione Granger,16.930,Chris Columbus,male,2.589,1492 Pictures,US
1,Harry Potter and the Philosopher's Stone,2001-11-16,305.856,7.9,21398,125000000,976475550,PG,Emma Watson,female,Hermione Granger,16.930,Chris Columbus,male,2.589,Warner Bros. Pictures,US
2,Harry Potter and the Philosopher's Stone,2001-11-16,305.856,7.9,21398,125000000,976475550,PG,Tom Felton,male,Draco Malfoy,10.918,Chris Columbus,male,2.589,1492 Pictures,US
3,Harry Potter and the Philosopher's Stone,2001-11-16,305.856,7.9,21398,125000000,976475550,PG,Tom Felton,male,Draco Malfoy,10.918,Chris Columbus,male,2.589,Heyday Films,GB
4,Harry Potter and the Philosopher's Stone,2001-11-16,305.856,7.9,21398,125000000,976475550,PG,Tom Felton,male,Draco Malfoy,10.918,Chris Columbus,male,2.589,Warner Bros. Pictures,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1262,Ghostbusters: Afterlife,2021-11-11,428.352,7.6,40,75000000,16000000,PG-13,Bill Murray,male,Dr. Peter Venkman,14.048,Jason Reitman,male,4.105,Columbia Pictures,US
1263,Ghostbusters: Afterlife,2021-11-11,428.352,7.6,40,75000000,16000000,PG-13,Bill Murray,male,Dr. Peter Venkman,14.048,Jason Reitman,male,4.105,The Montecito Picture Company,US
1264,Ghostbusters: Afterlife,2021-11-11,428.352,7.6,40,75000000,16000000,PG-13,Bill Murray,male,Dr. Peter Venkman,14.048,Jason Reitman,male,4.105,Bron Studios,CA
1265,Ghostbusters: Afterlife,2021-11-11,428.352,7.6,40,75000000,16000000,PG-13,Carrie Coon,female,Callie Spengler,11.375,Jason Reitman,male,4.105,Sony Pictures,US


In [7]:
# Create our features
X = pd.get_dummies(df.drop(columns='revenue'))

# Create our target
y = pd.get_dummies(df['revenue'])

In [8]:
X.describe()

,movie_id,movie_popularity,vote_average,vote_count,budget,actor_id,actor_popularity,director_id,director_popularity,studio_id,...,studio_name_Zero Gravity Management,studio_name_ufotable,origin_country_AU,origin_country_CA,origin_country_CN,origin_country_DE,origin_country_FR,origin_country_GB,origin_country_JP,origin_country_US
count,1165.000000,1165.000000,1165.000000,1165.000000,1.165000e+03,1.165000e+03,1165.000000,1.165000e+03,1165.000000,1165.000000,...,1165.000000,1165.000000,1165.000000,1165.000000,1165.00000,1165.000000,1165.000000,1165.000000,1165.000000,1165.000000
mean,366383.858369,779.779928,7.608927,9050.109013,1.377778e+08,1.766380e+05,18.331684,2.281539e+05,4.505484,20183.231760,...,0.001717,0.001717,0.005150,0.008584,0.01030,0.005150,0.016309,0.060086,0.008584,0.885837
std,241007.374882,1209.165937,0.643609,7240.485878,9.016638e+07,4.128657e+05,9.934214,4.451891e+05,3.665962,37406.106849,...,0.041416,0.041416,0.071611,0.092289,0.10101,0.071611,0.126715,0.237748,0.092289,0.318146
min,557.000000,173.026000,6.000000,40.000000,1.000000e+07,6.300000e+01,10.005000,9.560000e+02,0.728000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,271110.000000,256.019000,7.200000,2347.000000,7.000000e+07,5.469000e+03,11.672000,1.289100e+04,1.836000,174.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,370172.000000,391.146000,7.700000,6850.000000,1.160000e+08,1.760500e+04,14.904000,1.927200e+04,2.857000,444.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,550988.000000,672.641000,8.300000,15236.000000,2.000000e+08,7.396800e+04,21.049000,8.725700e+04,6.150000,12292.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.000000
max,791373.000000,8304.971000,8.400000,25788.000000,3.560000e+08,2.979464e+06,60.132000,1.932178e+06,16.390000,148495.000000,...,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
# Check the balance of our target values
y = df['revenue']
y.value_counts()

657000000     104
375540831      84
2797800564     56
2046239637     42
708000000      40
             ... 
61779301        4
148810604       3
112000000       2
467863133       2
136384442       2
Name: revenue, Length: 67, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({378328978: 3,
         167381210: 31,
         375540831: 59,
         168285000: 6,
         130000000: 11,
         895921036: 9,
         880166924: 27,
         657000000: 83,
         354264482: 8,
         976475550: 7,
         2046239637: 34,
         2797800564: 40,
         1341511219: 14,
         783766341: 18,
         4700000: 18,
         708000000: 33,
         757930663: 6,
         442000000: 24,
         25814306: 6,
         175302354: 10,
         938212738: 11,
         153000000: 12,
         331096766: 19,
         430238384: 8,
         821708551: 15,
         89500000: 5,
         348319861: 15,
         789804554: 15,
         16000000: 23,
         127000000: 9,
         629443428: 18,
         503063688: 7,
         155446362: 6,
         103966489: 6,
         34560577: 15,
         297372261: 4,
         721077945: 16,
         933959197: 7,
         1153296293: 12,
         708962323: 11,
         1518815515: 6,
         855013954: 12,
         

# Balanced Random Forest Classifier

In [11]:
# Resample the training data with the BalancedRandomForestClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.datasets import make_classification

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

brfc = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brfc.fit(X_train_scaled, y_train)

BalancedRandomForestClassifier(random_state=1)

In [12]:
# Calculated the balanced accuracy score
y_pred = brfc.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.015625

In [13]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[ 0,  0,  0, ...,  0,  6,  0],
       [ 0,  0,  0, ...,  0,  6,  0],
       [ 0,  0,  0, ...,  0, 10,  0],
       ...,
       [ 0,  0,  0, ...,  0,  8,  0],
       [ 0,  0,  0, ...,  0, 16,  0],
       [ 0,  0,  0, ...,  0,  6,  0]], dtype=int64)

In [14]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

     344931       0.00      0.00      1.00      0.00      0.00      0.00         6
    4700000       0.00      0.00      1.00      0.00      0.00      0.00         6
   16000000       0.00      0.00      1.00      0.00      0.00      0.00        10
   17635215       0.00      0.00      1.00      0.00      0.00      0.00         2
   19000000       0.00      0.00      1.00      0.00      0.00      0.00         3
   22039969       0.00      0.00      1.00      0.00      0.00      0.00         7
   24640084       0.00      0.00      1.00      0.00      0.00      0.00         1
   25814306       0.00      0.00      1.00      0.00      0.00      0.00         1
   34560577       0.00      0.00      1.00      0.00      0.00      0.00         3
   36964325       0.00      0.00      1.00      0.00      0.00      0.00         4
   45000000       0.00      0.00      1.00      0.00      0.00      0.00         1
   

In [15]:
# List the features sorted in descending order by feature importance
sorted(zip(brfc.feature_importances_, X.columns), reverse=True)

[(0.025384317214158188, 'director_id'),
 (0.0238353375108215, 'movie_popularity'),
 (0.023334111945494382, 'vote_count'),
 (0.023306996890246352, 'actor_popularity'),
 (0.02237575788622307, 'budget'),
 (0.022002161394125173, 'studio_id'),
 (0.02160403464931561, 'actor_id'),
 (0.021598210641601102, 'director_popularity'),
 (0.02100670187373825, 'vote_average'),
 (0.019895132532004242, 'movie_id'),
 (0.008133651072330135, 'actor_gender_female'),
 (0.007542849922302863, 'actor_gender_male'),
 (0.007531885577444375, 'ratings_PG-13'),
 (0.0067101656355894315, 'director_name_Chris Columbus'),
 (0.006644459455806146, 'director_gender_female'),
 (0.006126235388997326, 'ratings_PG'),
 (0.006084004542759987, 'studio_name_Marvel Studios'),
 (0.0056240774421971685, 'ratings_R'),
 (0.005622101395607256, 'origin_country_US'),
 (0.005588696451512426, 'director_gender_male'),
 (0.005418103732683224, 'studio_name_Warner Bros. Pictures'),
 (0.004684880770409747, 'director_name_Joe Russo'),
 (0.004419950

# Easy Ensemble AdaBoost Classifier

In [16]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier

eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)
eec.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [17]:
# Calculated the balanced accuracy score
y_pred = eec.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.4833891369047619

In [18]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0, 10, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  7,  0,  0],
       [ 0,  0,  0, ...,  0, 16,  0],
       [ 0,  0,  0, ...,  0,  0,  6]], dtype=int64)

In [19]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

     344931       0.00      0.00      1.00      0.00      0.00      0.00         6
    4700000       0.00      0.00      1.00      0.00      0.00      0.00         6
   16000000       1.00      1.00      1.00      1.00      1.00      1.00        10
   17635215       1.00      1.00      1.00      1.00      1.00      1.00         2
   19000000       1.00      0.67      1.00      0.80      0.82      0.64         3
   22039969       1.00      0.43      1.00      0.60      0.65      0.40         7
   24640084       1.00      1.00      1.00      1.00      1.00      1.00         1
   25814306       1.00      1.00      1.00      1.00      1.00      1.00         1
   34560577       0.00      0.00      1.00      0.00      0.00      0.00         3
   36964325       1.00      1.00      1.00      1.00      1.00      1.00         4
   45000000       1.00      1.00      1.00      1.00      1.00      1.00         1
   